In [10]:
from multiprocessing import Pool

In [11]:
Pool?

Signature: Pool(processes=None, initializer=None, initargs=(), maxtasksperchild=None)
Docstring: Returns a process pool object
File:      ~/anaconda3/envs/entropia_grafos/lib/python3.9/multiprocessing/context.py
Type:      method


In [8]:
def f(a, b, c):
    return a + b + c

In [9]:
f(*(1, 2, 3))

6

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scripts.Individual import Individual
from scripts.Parameters import mu, m, pa
from scripts.Polarity import polarity
from scripts.Memory import initialize_memory, probability_distribution, A
from scripts.Model import Model, evaluateModel, simulate
from scripts.Entropy import empirical_entropy, JSD
from scripts.Types import List
# from multiprocessing import Pool
from time import time

import numpy as np
import matplotlib.pyplot as plt
import random
import pickle
from copy import deepcopy

seed = 42

In [3]:
def list_sum(L1, L2):
    return [L1[k] + L2[k] for k in range(len(L1))]

def mean_lists(L: List[List]) -> List:
    # 1. soma todas as listas
    L0 = L[0]
    for L1 in L[1:]:
        L0 = list_sum(L0, L1)
    
    # 2. divide pelo número de listas
    N = len(L)
    for k in range(len(L0)):
        L0[k] = L0[k]/N
        
    return L0

In [4]:
def is_salient(code, P):
    return P[code] == max(P.values())

def is_in(code, P):
    return P[code] == 0

def compute_opinion_spread(M):
    D = deepcopy(A)
    for u in M.G.nodes():
        u_info = M.indInfo(u)
        for code in D.keys():
            if is_in(code, u_info.P):
                D[code] += 1
                
    N = len(M.G)
    for code in D.keys():
        D[code] = D[code]/N
        
    return D

def compute_opinion_fragmentation(M):
    D = deepcopy(A)
    for u in M.G.nodes():
        u_info = M.indInfo(u)
        for code in D.keys():
            if is_salient(code, u_info.P):
                D[code] += 1
                
    N = len(M.G)
    for code in D.keys():
        D[code] = D[code]/N
    
    return D

In [20]:
kappa = 10
num_repetitions = 5
seeds = [random.randint(1, 100) for i in range(num_repetitions)]
proportions = [(0.5, 0.5), (0.6, 0.4), (0.7, 0.3), (0.8, 0.2), (0.9, 0.1), (1.0, 0.0)]
# alpha = 0.5          # Proportion of individuals with tendency to increase polarity
# omega = 0.5          # Proportion of individuals with tendency to lower polarity
lambd = 10            # Polarization coefficient

T = 500

gamma = 1         # Confidence factor

H = {str(prop):None for prop in proportions}
pi = {str(prop):None for prop in proportions}
info_dist_stats = {str(prop):{code:[] for code in A.keys()} for prop in proportions}
N = 500

for alpha, omega in proportions:
    
    print("Inicializando o modelo com os parâmetros:")
    print("N = {} - pa = {} - mu = {} - m = {} - kappa = {} - lambda = {} - alpha = {} - omega = {} - gamma = {}".format(N, pa, mu, m, kappa, lambd, alpha, omega, gamma))
    model = Model(N, pa, mu, m, kappa, lambd, alpha, omega, gamma, 0, False)
    
    H[str((alpha, omega))] = []
    pi[str((alpha, omega))] = []
    D = deepcopy(A)
    for code in D.keys():
        D[code] = []
    
    for rep in range(num_repetitions):
        H_rep_temp    = []
        Pi_rep_temp   = []
        # Especifica um valor inicial para o gerador de números pseudo-aleatórios e inicializa/reiniciar o modelo com esse gerador.
        seed = seeds[rep]
        model.seed = seed
        model.initialize_model_info()
        
        print("Iniciando Repetição N. {}".format(rep))
        elapsedTime = 0
        for i in range(T):
            elapsedTime += simulate(model)
            H_rep_temp.append(model.H)
            Pi_rep_temp.append(model.pi)
            D_temp = compute_opinion_fragmentation(model)
            for code in D.keys():
                D[code].append(D_temp[code])
            
        # Apende aos dados vinculados ao parâmetro "k" os valores de todas as repetições
        print("Finalizada a Repetição N. {} em {} segundos.".format(rep, elapsedTime))
        H[str((alpha, omega))].append(H_rep_temp)
        pi[str((alpha, omega))].append(Pi_rep_temp)
        for code in D.keys():   
            info_dist_stats[str((alpha, omega))][code].append(D[code])
    
# Tira a média das simulações para cada k
for prop in proportions:
    H[str(prop)] = mean_lists(H[str(prop)])
    pi[str(prop)] = mean_lists(pi[str(prop)])
    for code in info_dist_stats[str(prop)].keys():
        info_dist_stats[str(prop)][code] = mean_lists(info_dist_stats[str(prop)][code])

Inicializando o modelo com os parâmetros:
N = 500 - pa = 2 - mu = 360 - m = 5 - kappa = 10 - lambda = 10 - alpha = 0.5 - omega = 0.5 - gamma = 1
Iniciando Repetição N. 0
Finalizada a Repetição N. 0 em 2.84486233393351 segundos.
Iniciando Repetição N. 1
Finalizada a Repetição N. 1 em 2.812279184659324 segundos.
Iniciando Repetição N. 2
Finalizada a Repetição N. 2 em 2.8471983075141893 segundos.
Iniciando Repetição N. 3
Finalizada a Repetição N. 3 em 2.912819449106853 segundos.
Iniciando Repetição N. 4
Finalizada a Repetição N. 4 em 2.884556130568184 segundos.
Inicializando o modelo com os parâmetros:
N = 500 - pa = 2 - mu = 360 - m = 5 - kappa = 10 - lambda = 10 - alpha = 0.6 - omega = 0.4 - gamma = 1
Iniciando Repetição N. 0
Finalizada a Repetição N. 0 em 2.867674044768017 segundos.
Iniciando Repetição N. 1
Finalizada a Repetição N. 1 em 2.7899048328399654 segundos.
Iniciando Repetição N. 2
Finalizada a Repetição N. 2 em 3.3208956638971956 segundos.
Iniciando Repetição N. 3
Finalizada 

In [19]:
stats = {'H':H, 'pi':pi}
name = 'Stats - (alpha, omega) in {} - lambda = {} - kappa = {}'.format(proportions, lambd, kappa)
with open ('Outputs/'+name, 'wb') as stats_file:
    pickle.dump(stats, stats_file)
    
name = 'Opinion Fragmentation - (alpha, omega) in {} - lambda = {} - kappa = {}'.format(proportions, lambd, kappa)
with open('Outputs/'+name, 'wb') as stats_file:
    pickle.dump(info_dist_stats, stats_file)

In [85]:
kappa = 0
# kappa = 10
# kappa = 15
num_repetitions = 1
seeds = [random.randint(1, 100) for i in range(num_repetitions)]
# proportions = [(0.5, 0.5), (0.6, 0.4), (0.7, 0.3), (0.8, 0.2), (0.9, 0.1), (1.0, 0.0)]
# proportions = [(0.6, 0.0), (0.5, 0.1), (0.4, 0.2), (0.3, 0.3)]
proportions = [(0.7, 0.0), (0.6, 0.1), (0.5, 0.2), (0.4, 0.3)]
# alpha = 0.5          # Proportion of individuals with tendency to increase polarity
# omega = 0.5          # Proportion of individuals with tendency to lower polarity
# lambd = 0
lambd = 0.1
# lambd = 0.2
# lambd = 0.3
# lambd = 0.4
# lambd = 0.5

T = 1000

gamma = 0         # Confidence factor
# gamma = 5
# gamma = -5

H = {str(prop):None for prop in proportions}
pi = {str(prop):None for prop in proportions}
info_dist_stats = {str(prop):{code:[] for code in A.keys()} for prop in proportions}
N = 500

for alpha, omega in proportions:
    
    print("Inicializando o modelo com os parâmetros:")
    print("N = {} - pa = {} - mu = {} - m = {} - kappa = {} - lambda = {} - alpha = {} - omega = {} - gamma = {}".format(N, pa, mu, m, kappa, lambd, alpha, omega, gamma))
    model = Model(N, pa, mu, m, kappa, lambd, alpha, omega, gamma, 0, False)
    
    H[str((alpha, omega))] = []
    pi[str((alpha, omega))] = []
    
    for rep in range(num_repetitions):
        H_rep_temp    = []
        Pi_rep_temp   = []
        D = deepcopy(A)
        for code in D.keys():
            D[code] = []
        # Especifica um valor inicial para o gerador de números pseudo-aleatórios e inicializa/reiniciar o modelo com esse gerador.
        seed = seeds[rep]
        model.seed = seed
        model.initialize_model_info()
        
        print("Iniciando Repetição N. {}".format(rep))
        elapsedTime = 0
        for i in range(T):
            elapsedTime += simulate(model)
            H_rep_temp.append(model.H)
            Pi_rep_temp.append(model.pi)
            D_temp = compute_opinion_fragmentation(model)
            for code in D.keys():
                D[code].append(D_temp[code])
            
        # Apende aos dados vinculados ao parâmetro "k" os valores de todas as repetições
        print("Finalizada a Repetição N. {} em {} segundos.".format(rep, elapsedTime))
        H[str((alpha, omega))].append(H_rep_temp)
        pi[str((alpha, omega))].append(Pi_rep_temp)
        for code in D.keys():   
            info_dist_stats[str((alpha, omega))][code].append(D[code])
    
# Tira a média das simulações para cada k
for prop in proportions:
    H[str(prop)] = mean_lists(H[str(prop)])
    pi[str(prop)] = mean_lists(pi[str(prop)])
    for code in info_dist_stats[str(prop)].keys():
        info_dist_stats[str(prop)][code] = mean_lists(info_dist_stats[str(prop)][code])

Inicializando o modelo com os parâmetros:
N = 500 - pa = 2 - mu = 360 - m = 5 - kappa = 0 - lambda = 0.1 - alpha = 0.7 - omega = 0.0 - gamma = 0
Iniciando Repetição N. 0
Finalizada a Repetição N. 0 em 6.3314383308092745 segundos.
Inicializando o modelo com os parâmetros:
N = 500 - pa = 2 - mu = 360 - m = 5 - kappa = 0 - lambda = 0.1 - alpha = 0.6 - omega = 0.1 - gamma = 0
Iniciando Repetição N. 0
Finalizada a Repetição N. 0 em 7.15025939146678 segundos.
Inicializando o modelo com os parâmetros:
N = 500 - pa = 2 - mu = 360 - m = 5 - kappa = 0 - lambda = 0.1 - alpha = 0.5 - omega = 0.2 - gamma = 0
Iniciando Repetição N. 0
Finalizada a Repetição N. 0 em 7.2572814742724026 segundos.
Inicializando o modelo com os parâmetros:
N = 500 - pa = 2 - mu = 360 - m = 5 - kappa = 0 - lambda = 0.1 - alpha = 0.4 - omega = 0.3 - gamma = 0
Iniciando Repetição N. 0
Finalizada a Repetição N. 0 em 8.162457346916199 segundos.


In [84]:
stats = {'H':H, 'pi':pi}
name = 'Stats - (alpha, omega) in {} - lambda = {} - kappa = {}'.format(proportions, lambd, kappa)
with open ('Outputs/'+name, 'wb') as stats_file:
    pickle.dump(stats, stats_file)
    
name = 'Opinion Fragmentation - (alpha, omega) in {} - lambda = {} - kappa = {}'.format(proportions, lambd, kappa)
with open('Outputs/'+name, 'wb') as stats_file:
    pickle.dump(info_dist_stats, stats_file)